In [14]:
import MetaTrader5 as mt5  # pip install MetaTrader5
import pandas as pd  # pip install pandas
import plotly.express as px  # pip install plotly
from datetime import datetime, timedelta
from mt5_config import login_number, path

In [15]:
result = mt5.initialize(path)
if login_number == mt5.account_info().login and result is True:
    print('Connection with MT5 established')
else:
    print('Connection failed')
    mt5.shutdown()

Connection with MT5 established


In [4]:
result = mt5.initialize()
result

True

In [8]:
volume = 1.0 # float
action = mt5.TRADE_ACTION_DEAL
order_type = mt5.ORDER_TYPE_BUY

stop_loss = 1.0  # set to 0.0 if you don't want SL
take_profit = 1.2  # set to 0.0 if you don't want TP

def get_market_price(symbol, type):
    if type == mt5.ORDER_TYPE_BUY:
        return mt5.symbol_info(symbol).ask
    elif type == mt5.ORDER_TYPE_SELL:
        return mt5.symbol_info(symbol).bid

request = {
    "action": action,
    "symbol": "EURUSD",
    "volume": volume,  # float
    "type": mt5.ORDER_TYPE_BUY,
    "price": get_market_price("EURUSD", order_type),
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 1,
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10009, deal=1095338128, order=1338143091, volume=1.0, price=1.1565400000000001, bid=1.1565400000000001, ask=1.1565400000000001, comment='Request executed', request_id=601666035, retcode_external=0, request=TradeRequest(action=1, magic=1, order=0, symbol='EURUSD', volume=1.0, price=1.1565400000000001, stoplimit=0.0, sl=1.0, tp=1.2, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))

In [6]:
# modify SL/TP

stop_loss = 0.5  # set to 0.0 if you don't want SL
take_profit = 2.0  # set to 0.0 if you don't want TP

positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

request = {
    'action': mt5.TRADE_ACTION_SLTP,
    'position': pos1.ticket,
    'sl': stop_loss,
    'tp': take_profit
}

res = mt5.order_send(request)
res

open positions (TradePosition(ticket=1338075845, time=1764096060, time_msc=1764096060797, time_update=1764096060, time_update_msc=1764096060797, type=0, magic=0, identifier=1338075845, reason=0, volume=1.0, price_open=1.15584, sl=0.0, tp=0.0, price_current=1.15644, swap=0.0, profit=60.0, symbol='EURUSD', comment='', external_id=''), TradePosition(ticket=1338109306, time=1764097068, time_msc=1764097068463, time_update=1764097068, time_update_msc=1764097068463, type=0, magic=1, identifier=1338109306, reason=3, volume=1.0, price_open=1.15514, sl=1.0, tp=1.2, price_current=1.15644, swap=0.0, profit=130.0, symbol='EURUSD', comment='python market or', external_id=''))


OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=601666033, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=0.5, tp=2.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=1338075845, position_by=0))

In [7]:
positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

def reverse_type(type):
    # to close a buy positions, you must perform a sell position and vice versa
    if type == mt5.ORDER_TYPE_BUY:
        return mt5.ORDER_TYPE_SELL
    elif type == mt5.ORDER_TYPE_SELL:
        return mt5.ORDER_TYPE_BUY


def get_close_price(symbol, type):
    if type == mt5.ORDER_TYPE_BUY:
        return mt5.symbol_info(symbol).bid
    elif type == mt5.ORDER_TYPE_SELL:
        return mt5.symbol_info(symbol).ask

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "position": pos1.ticket,
    "symbol": pos1.symbol,
    "volume": pos1.volume,
    "type": reverse_type(pos1.type),
    "price":get_close_price(pos1.symbol, pos1.type),
    "deviation": 20,
    "magic": 0,
    "comment": "python close order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

open positions (TradePosition(ticket=1338075845, time=1764096060, time_msc=1764096060797, time_update=1764096060, time_update_msc=1764096060797, type=0, magic=0, identifier=1338075845, reason=0, volume=1.0, price_open=1.15584, sl=0.5, tp=2.0, price_current=1.15648, swap=0.0, profit=64.0, symbol='EURUSD', comment='', external_id=''), TradePosition(ticket=1338109306, time=1764097068, time_msc=1764097068463, time_update=1764097068, time_update_msc=1764097068463, type=0, magic=1, identifier=1338109306, reason=3, volume=1.0, price_open=1.15514, sl=1.0, tp=1.2, price_current=1.15648, swap=0.0, profit=134.0, symbol='EURUSD', comment='python market or', external_id=''))


OrderSendResult(retcode=10009, deal=1095326077, order=1338130040, volume=1.0, price=1.15648, bid=1.15648, ask=1.15648, comment='Request executed', request_id=601666034, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD', volume=1.0, price=1.15648, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python close order', position=1338075845, position_by=0))

In [9]:
perc_close = 0.5

positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

def reverse_type(type):
    # to close a buy positions, you must perform a sell position and vice versa
    if type == 0:
        return 1
    elif type == 1:
        return 0


def get_close_price(symbol, type):
    if type == 0:
        return mt5.symbol_info(symbol).bid
    elif type == 1:
        return mt5.symbol_info(symbol).ask

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "position": pos1.ticket,
    "symbol": pos1.symbol,
    "volume": round(pos1.volume * perc_close, 2),  # closing only half
    "type": reverse_type(pos1.type),
    "price":get_close_price(pos1.symbol, pos1.type),
    "deviation": 20,
    "magic": 0,
    "comment": "python partial close order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

open positions (TradePosition(ticket=1338109306, time=1764097068, time_msc=1764097068463, time_update=1764097068, time_update_msc=1764097068463, type=0, magic=1, identifier=1338109306, reason=3, volume=1.0, price_open=1.15514, sl=1.0, tp=1.2, price_current=1.1562000000000001, swap=0.0, profit=106.0, symbol='EURUSD', comment='python market or', external_id=''), TradePosition(ticket=1338143091, time=1764097532, time_msc=1764097532559, time_update=1764097532, time_update_msc=1764097532559, type=0, magic=1, identifier=1338143091, reason=3, volume=1.0, price_open=1.1565400000000001, sl=1.0, tp=1.2, price_current=1.1562000000000001, swap=0.0, profit=-34.0, symbol='EURUSD', comment='python market or', external_id=''))


OrderSendResult(retcode=10009, deal=1095339913, order=1338145059, volume=0.5, price=1.1562000000000001, bid=1.1562000000000001, ask=1.1562000000000001, comment='Request executed', request_id=601666036, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSD', volume=0.5, price=1.1562000000000001, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python partial close order', position=1338109306, position_by=0))

In [10]:
symbol = "EURUSD"
volume = 0.1

action = mt5.TRADE_ACTION_PENDING
order_type = mt5.ORDER_TYPE_BUY_LIMIT  #  mt5.ORDER_TYPE_SELL_LIMIT
limit_price = 1.0

stop_loss = 0.9  # set to 0.0 if you don't want SL
take_profit = 1.2  # set to 0.0 if you don't want TP

request = {
    "action": action,
    "symbol": "EURUSD",
    "volume": 0.1,  # float
    "type": order_type,
    "price": limit_price,
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 0,
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10009, deal=0, order=1338149541, volume=0.1, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=601666037, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.0, stoplimit=0.0, sl=0.9, tp=1.2, deviation=20, type=2, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))

In [11]:
symbol = "EURUSD"
volume = 0.1

action = mt5.TRADE_ACTION_PENDING
order_type = mt5.ORDER_TYPE_BUY_STOP  # mt5.ORDER_TYPE_SELL_STOP
stop_price = 1.2

stop_loss = 1.0  # set to 0.0 if you don't want SL
take_profit = 1.3  # set to 0.0 if you don't want TP

request = {
    "action": action,
    "symbol": "EURUSD",
    "volume": 0.1,  # float
    "type": order_type,
    "price": stop_price,
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 0,
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10009, deal=0, order=1338159089, volume=0.1, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=601666041, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.2, stoplimit=0.0, sl=1.0, tp=1.3, deviation=20, type=4, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))

In [12]:
order = 1338159089
action = mt5.TRADE_ACTION_MODIFY

request = {
    'action': action,
    'order': order,
    'price': 1.25,
    'sl': 1.1,
    'tp': 1.3
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10009, deal=0, order=1338159089, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=601666042, retcode_external=0, request=TradeRequest(action=7, magic=0, order=1338159089, symbol='', volume=0.0, price=1.25, stoplimit=0.0, sl=1.1, tp=1.3, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

In [13]:
pending_orders = mt5.orders_get()
print('open pending orders', pending_orders)

order1 = pending_orders[0]

request = {
    'action': mt5.TRADE_ACTION_REMOVE,
    'order': order1.ticket
}

mt5.order_send(request)

open pending orders (TradeOrder(ticket=1338071751, time_setup=1764095899, time_setup_msc=1764095899556, time_done=0, time_done_msc=0, time_expiration=0, type=3, type_time=0, type_filling=2, state=1, magic=0, position_id=0, position_by_id=0, reason=0, volume_initial=1.0, volume_current=1.0, price_open=1.16014, sl=0.0, tp=0.0, price_current=1.15653, price_stoplimit=0.0, symbol='EURUSD', comment='', external_id=''), TradeOrder(ticket=1338073018, time_setup=1764095957, time_setup_msc=1764095957238, time_done=0, time_done_msc=0, time_expiration=0, type=2, type_time=0, type_filling=2, state=1, magic=0, position_id=0, position_by_id=0, reason=0, volume_initial=1.0, volume_current=1.0, price_open=1.15271, sl=0.0, tp=0.0, price_current=1.15653, price_stoplimit=0.0, symbol='EURUSD', comment='', external_id=''), TradeOrder(ticket=1338149541, time_setup=1764097710, time_setup_msc=1764097710120, time_done=0, time_done_msc=0, time_expiration=0, type=2, type_time=0, type_filling=2, state=1, magic=0, 

OrderSendResult(retcode=10009, deal=0, order=1338071751, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=601666043, retcode_external=0, request=TradeRequest(action=8, magic=0, order=1338071751, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))